In [112]:
import numpy as np
import heapq
import math

In [113]:
def h_manhatten(cur_s, end_s):
    dis = 0
    for i, x in np.ndenumerate(cur_s):
        if x == 0:
            continue
        ind = np.where(end_s == x)
        dis += abs(i[0] - ind[0][0]) + abs(i[1] - ind[1][0])
    return dis

In [114]:
def possible_moves(cur_s):
    moves = []
    zero_ind = np.where(cur_s == 0)
    r = zero_ind[0][0]
    c = zero_ind[1][0]
    dr = [0, 0, -1, 1]
    dc = [-1, 1, 0, 0]
    move = ['Left', 'Right', 'Up', 'Down']
    for i in range(4):
        nr = r + dr[i]
        nc = c + dc[i]
        if 0 <= nr <= 2 and 0 <= nc <= 2:
            new_state = cur_s.copy()
            v1 = new_state[r][c]
            v2 = new_state[nr][nc]
            new_state[r][c] = v2
            new_state[nr][nc] = v1
            moves.append((new_state, move[i]))
    return moves

In [115]:
def path_track(came_from, current):
    path = []
    while current in came_from.keys():
        prev, move = came_from[current]
        path.append(move)
        current = prev
    path.reverse()
    return path

In [116]:
def is_solvable(state):
    arr = [x for x in state if x != 0]
    inv = 0
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            if arr[i] > arr[j]:
                inv += 1
    return inv % 2 == 0

In [117]:
def pretty_print(state):
    for row in state:
        print(' '.join(str(x) if x != 0 else '_' for x in row))
    print()

In [118]:
# --- A* Search ---
def a_star(start_s, end_s):
    start_arr = np.array(start_s)
    end_arr = np.array(end_s)

    if not is_solvable(start_arr.flatten().tolist()):
        raise ValueError("This puzzle configuration is not solvable.")

    start_key = tuple(start_arr.flatten())
    end_key = tuple(end_arr.flatten())

    g_score = {start_key: 0}
    f_start = h_manhatten(start_arr, end_arr)
    open_heap = [(f_start, 0, start_key)]
    came_from = {}
    closed_set = set()

    tie = 1

    while open_heap:
        f, _, current_key = heapq.heappop(open_heap)
        if current_key in closed_set:
            continue

        current_arr = np.array(current_key).reshape(3, 3)

        if np.array_equal(current_arr, end_arr):
            moves = path_track(came_from, current_key)
            return moves

        closed_set.add(current_key)

        for nbr_arr, move in possible_moves(current_arr):
            nbr_key = tuple(nbr_arr.flatten())
            if nbr_key in closed_set:
                continue

            tentative_g = g_score[current_key] + 1
            if tentative_g < g_score.get(nbr_key, math.inf):
                g_score[nbr_key] = tentative_g
                f_score = tentative_g + h_manhatten(nbr_arr, end_arr)
                tie += 1
                heapq.heappush(open_heap, (f_score, tie, nbr_key))
                came_from[nbr_key] = (current_key, move)

    raise RuntimeError("A* failed to find a solution.")

In [119]:
start_state = [
        [1, 0, 2],
        [3, 4, 6],
        [7, 5, 8]
]

goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

print("Start state:")
pretty_print(start_state)
print("Goal state:")
pretty_print(goal_state)

try:
    moves= a_star(start_state, goal_state)
    print(f"Solution found in {len(moves)} moves.")
    print("Moves sequence:", moves)

    cur = np.array(start_state)
    for i, m in enumerate(moves, 1):
        for nbr_arr, mv in possible_moves(cur):
            if mv == m:
                cur = nbr_arr
                break
        print(f"After move {i}: {m}")
        pretty_print(cur.tolist())

except ValueError as e:
    print(e)

Start state:
1 _ 2
3 4 6
7 5 8

Goal state:
1 2 3
4 5 6
7 8 _

Solution found in 15 moves.
Moves sequence: ['Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Down', 'Right']
After move 1: Right
1 2 _
3 4 6
7 5 8

After move 2: Down
1 2 6
3 4 _
7 5 8

After move 3: Left
1 2 6
3 _ 4
7 5 8

After move 4: Left
1 2 6
_ 3 4
7 5 8

After move 5: Up
_ 2 6
1 3 4
7 5 8

After move 6: Right
2 _ 6
1 3 4
7 5 8

After move 7: Down
2 3 6
1 _ 4
7 5 8

After move 8: Right
2 3 6
1 4 _
7 5 8

After move 9: Up
2 3 _
1 4 6
7 5 8

After move 10: Left
2 _ 3
1 4 6
7 5 8

After move 11: Left
_ 2 3
1 4 6
7 5 8

After move 12: Down
1 2 3
_ 4 6
7 5 8

After move 13: Right
1 2 3
4 _ 6
7 5 8

After move 14: Down
1 2 3
4 5 6
7 _ 8

After move 15: Right
1 2 3
4 5 6
7 8 _

